In [1]:
#Imports
import redis
import urllib.request
import json
import numpy as np
import pandas as pd
import schedule
import time
import boto3
import datetime

import mysql.connector
from sqlalchemy import create_engine
from sqlalchemy import inspect
from sqlalchemy import MetaData
from sqlalchemy import Table

kinesis = boto3.client('kinesis', region_name='us-east-1')


#Connect to Redis-DataStore
REDIS = redis.Redis(host='data_store')

In [2]:
%env AWS_SECRET_ACCESS_KEY

'82pgo1H+eurVJr548ZNAuYbqzCewoSojQ4B90TWJ'

In [3]:
kinesis.list_streams()

{'HasMoreStreams': False,
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '41',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Thu, 01 Feb 2018 20:24:39 GMT',
   'server': 'Apache-Coyote/1.1',
   'x-amz-id-2': 'pz6qHS6uc/6BEnnsgGNShCUnomkR6OXZpx3ylbBHuLIiB/ucBUPsDmfbJO68PTVGrEMfAK2Q8WHWmUp2vKYdkZAVHoZoFzODa1UbIJj6TJ4=',
   'x-amzn-requestid': 'ceea6709-ff67-48e1-9c07-1ece970d3eb4'},
  'HTTPStatusCode': 200,
  'RequestId': 'ceea6709-ff67-48e1-9c07-1ece970d3eb4',
  'RetryAttempts': 0},
 'StreamNames': []}

### Redis Test

In [4]:
REDIS.get('my_incrementer')

b'42'

In [5]:
#To pass a dictionary:
import numpy as np
import json

model_params = {
    'C': list(np.logspace(-3,3,7)),
    'penalty': 'l1',
    'solver' : 'newton-cg'
}

REDIS.set('model_params', json.dumps(model_params))

True

#### Build Company dataframe

In [20]:
companies = json.loads(REDIS.get('companies').decode())
stock_tickers = list(companies.keys())
stock_tickers

['AAPL', 'FB', 'GOOG']

In [17]:
#Manager Log
# Need to send Number of Companies, Rate
now = datetime.datetime.now()
now

datetime.datetime(2018, 2, 1, 20, 29, 26, 66148)

### IEX Code -------------------------------------------------------
1. Define the stock attributes
2. Define functions: filter attributes, fetch stock data, log events
3. Read company list from the redis datastore
4. Setup schedule
5. Fetch data at scheduled event.

In [7]:
#Imports
import redis
import urllib.request
import json
import numpy as np
import pandas as pd
import schedule
import time
import boto3
from datetime import date, datetime

import mysql.connector
from sqlalchemy import create_engine
from sqlalchemy import inspect
from sqlalchemy import MetaData
from sqlalchemy import Table

kinesis = boto3.client('kinesis', region_name='us-east-1')


#Connect to Redis-DataStore
REDIS = redis.Redis(host='data_store')

In [2]:
User = %env DB_USER
password = %env DB_PWD
dbname = %env DB_NAME

In [14]:
############ Define Functions #################################
attributes = ['latestUpdate',
             'companyName',
             'latestPrice',
              'latestVolume',
              'marketCap',
              'open',
              'previousClose',
              'sector',
              'high',
              'low',
              'ytdChange',
              'peRatio',
              'week52High',
              'week52Low'
               ]

def filter_stock_attributes(data):
    output = {}
    #Choose filter attributes
    for element in attributes:
        output[element]=data[element]
    
    #Convert time
    ctime = output['latestUpdate']/1000
    new_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(ctime))
    output['latestUpdate']= new_time
    
    return output

#Create function to fetch the stock data. This is to prepare for the schedule.
def fetch_stock_data(stocks=[]):
    #Need to read the dataOn flag. 
    
    for ticker in stocks:
        url = "https://api.iextrading.com/1.0/stock/{}/quote".format(ticker)
        response = urllib.request.urlopen(url)
        str_response = response.read().decode('utf-8')
        obj = json.loads(str_response)
        filtered = filter_stock_attributes(obj)
        #Add counter to count stocks. 
        REDIS.incr('IEX_Stock_Count')
        #<----- Insert to Kinesis Stream ------->
        #response = kinesis.put_record(StreamName="IEX_Stream", Data=json.dumps(filtered), PartitionKey="partitionkey")
        print('---------------------------------')
        #print(response)
        print(filtered)

#Setup Log

global past_stock_count
past_stock_count = 0

#Setup Database
# engine = create_engine('mysql+mysqlconnector://{}:{}@db:3306/{}'.format(User, password, dbname), echo=False)
# conn = engine.connect()

# #Create table object
# meta = MetaData(engine,reflect=True)
# table = meta.tables['log']

#Serialize datetime.
def json_serial(obj):
    """JSON serializer for objects not serializable by default json code"""

    if isinstance(obj, (datetime, date)):
        return obj.isoformat()
    raise TypeError ("Type %s not serializable" % type(obj))

########### Redis Subscription setup ########################
queue = REDIS.pubsub()
#Subscribe to the queues one for the events and one for the log
queue.subscribe('event_queue')
queue.subscribe('log_queue')

#Code to log to the event queue
def send_event(source, kind, message):
    event_time = datetime.now()
    event_time = json_serial(event_time)
    event = {
            "event_time": event_time,
            "source": source,
            "kind" : kind,
            "message" : message
            }
    payload = json.dumps(event)
    REDIS.publish('event_queue', payload)

def send_log(source, current_count, count_diff):
    log_time = datetime.now()
    log_time = json_serial(log_time)
    log = {
            "log_time": log_time,
            "source": source,
            "current_count" : current_count,
            "count_diff" : count_diff
            }
    payload = json.dumps(log)
    REDIS.publish('log_queue', payload)


#Send the log data to the Redis channel.
def log():
    #Need to log: Time, Source, Current Count, Count Diff
    #now = datetime.datetime.now()
    current_stock_count = int(REDIS.get('IEX_Stock_Count'))    
     
    global past_stock_count
    stock_count_diff = current_stock_count - past_stock_count
    past_stock_count = current_stock_count
    
    #Send the log event
    send_log(source='IEX',current_count = current_stock_count, count_diff=stock_count_diff)
    
    #Need to log these items to a database.
#     ins = table.insert().values(
#             log_time = now,
#             Source = 'IEX',
#             Current_Count=current_stock_count,
#             Count_Diff=stock_count_diff
#                )
#     conn.execute(ins)
    print('Logged Data')

In [15]:
#Read Data-Store
companies = json.loads(REDIS.get('companies').decode())
stock_tickers = list(companies.keys())
REDIS.set('IEX_Stock_Count', 0)

#Setup Schedule
schedule.clear()
schedule.every(30).seconds.do(fetch_stock_data, stocks=stock_tickers)
schedule.every(35).seconds.do(log)

#Send Start event
send_event('IEX', 'Activity', 'Data Source Started')

#Execute
while True:
    schedule.run_pending()
    time.sleep(1)

---------------------------------
{'latestUpdate': '2018-02-02 21:00:00', 'companyName': 'Apple Inc.', 'latestPrice': 160.5, 'latestVolume': 86339265, 'marketCap': 815604985500, 'open': 166.34, 'previousClose': 167.78, 'sector': 'Technology', 'high': 166.8, 'low': 160.1, 'ytdChange': -0.06826889585510271, 'peRatio': 17.45, 'week52High': 180.1, 'week52Low': 128.9}
---------------------------------
{'latestUpdate': '2018-02-02 21:00:00', 'companyName': 'Facebook Inc.', 'latestPrice': 190.28, 'latestVolume': 25768287, 'marketCap': 552763544232, 'open': 192.1, 'previousClose': 193.09, 'sector': 'Technology', 'high': 194.21, 'low': 189.98, 'ytdChange': 0.04883695292691001, 'peRatio': 30.94, 'week52High': 195.32, 'week52Low': 130.3}
---------------------------------
{'latestUpdate': '2018-02-02 21:00:00', 'companyName': 'Alphabet Inc.', 'latestPrice': 1111.9, 'latestVolume': 4836050, 'marketCap': 772529217700, 'open': 1121.95, 'previousClose': 1167.7, 'sector': 'Technology', 'high': 1123.07,

KeyboardInterrupt: 